<a href="https://colab.research.google.com/github/thad75/TP-ENSEA-ELEVE/blob/main/2A/TP%202021%202022/Option%20IA/TD/RecSysTD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip "/content/ml-20m.zip"
!pip install pytorch-lightning
import os
import shutil

import pandas as pd
import pycocotools
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from IPython.display import clear_output, display
from pytorch_lightning import loggers as pl_loggers
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import MNIST

pd.set_option("expand_frame_repr", False)


--2023-03-04 10:44:58--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  75.5MB/s    in 2.5s    

2023-03-04 10:45:01 (75.5 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  /content/ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 6.8 MB/

# Hands-On : Recommending an artist

Given the following data, we want to recommend a list of artists from our existing database to the listener. Using similarity computation, recommend in the decreasing order the artists that suits listerners' taste

In [ ]:
data = {
    'Artist': ['Celine Dion', 'Fred Again', 'Elton John', 'Black Eyed Peas', 'BTS', 'Anirudh', 'A.R Rahman', 'User'],
    'International': [1, 1, 1, 1, 1, 1, 1, 1],
    'Sings in French': [1, 0, 0, 0, 0, 0, 0, 0],
    'Versatility': [0, 0, 0, 0, 0, 1, 1, 1],
    'Danceability': [0, 1, 0, 1, 1, 1, 1, 0],
    'Makes Film Music (BGM)': [0, 0, 1, 0, 0, 1, 1, 1],
    'Won Awards': [1, 1, 1, 1, 1, 1, 1, 1],
    'Recent Hit': [0, 1, 0, 0, 1, 1, 1, 0]
}

df = pd.DataFrame(data)
df = df.set_index('Artist')

# TODO : Print the datafmale
print(df)

                 International  Sings in French  Versatility  Danceability  Makes Film Music (BGM)  Won Awards  Recent Hit
Artist                                                                                                                    
Celine Dion                  1                1            0             0                       0           1           0
Fred Again                   1                0            0             1                       0           1           1
Elton John                   1                0            0             0                       1           1           0
Black Eyed Peas              1                0            0             1                       0           1           0
BTS                          1                0            0             1                       0           1           1
Anirudh                      1                0            1             1                       1           1           1
A.R Rahman      

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances


# TODO : Extract users' likings (check loc method from df)
user= df.loc[['User']]
other_line = df.loc[['Celine Dion']]
keys = [key for key in df.index if 'User' not in key]

# TODO : Define a Similarity Function (Cosine or Distance will be enough)
def similarity_function(l1, l2):
  return manhattan_distances(l1,l2)
# TODO : Compare each artists to the user's taste and print the list of recommendable artist.
values = []

for key in keys: 

  value = similarity_function(user, df.loc[[key]])
  values.append((key, value))
distance_df = pd.DataFrame(values, columns=["key", "similarity"])
distance_df.set_index('key')
print(distance_df)

               key similarity
0      Celine Dion    [[3.0]]
1       Fred Again    [[4.0]]
2       Elton John    [[1.0]]
3  Black Eyed Peas    [[3.0]]
4              BTS    [[4.0]]
5          Anirudh    [[2.0]]
6       A.R Rahman    [[2.0]]


# Hands-on: Feature Extraction and Hashing Tricks

In [2]:
from sklearn.feature_extraction import FeatureHasher

# Creating array of dicts
data = [
    {'dog': -1, 'cat': 2, 'elephant': 4},
    {'dog': 2, 'run': 5, 'cat':-7}
]

h = FeatureHasher(n_features=150)
h.transform(data).toarray()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -4.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       

# Recommendation Systems

Using MovieLens Dataset, build a recommendation system. Feel free for your model. We suggest you to inspire yourself from the NCF Model. This mini lab is **voluntarily sparse**. As future engineers,you'll need to go fetch informations everywhere in order to create and train your model. Don't hesitate to ask questions ! A live correction will be given.

Goal of this mini Lab :   
*  **Be lost**
* Discover new frameworks and read documentation
* Explore Datas and think
* Create and Train a model using Pytorch Lightning

# MovieLens Dataset

As a Data Scientist, your work is to understand the data your working with. Using Panda Dataframe have a look at the ratings.csv and movies.csv files and answer the following questions.

* What are the features of both csv files ?
* To you, what could be the most important csv file ?
* How many movies are there ?
* How many users rated movies ? 
* How users did rate the movies ? 
* What's the span of the ratings ? Using matplotlib or seaborn, plot the ratings for a given movie.
* Can we merge both tables ? if yes what key should be used to merge them ?
* What are the differents genres of the movies ?
* Are there outliers ?

Documentation available on : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

# Some Changes

For the rest of the lab, we will only use .05% of the dataset as it it too much for this notebook ( RIP Google ). Run the following cell for the rating dataframe. It will replace the existing rating dataset by a splitted dataset corresponding to 0.05% of the dataset

In [ ]:
ratings, _ = train_test_split(ratings, test_size = 0.95)

# Create your Dataset

At this point, you should have a good understanding of the given model. 
The given Dataset is a csv file. As we work with Tensors and Pytorch, we need to convert the Dataset to something readable by a Pytorch Model. Define a Dataset class that inherits from Dataset that returns a dictionnary {user, item, ratings}. The skeleton is given.
Don't forget to create your train, test, validation datasets.

Documentation available on : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class MovieLensDataset(Dataset):

  def __init__(self,dataframe):
      self.dataframe = 

  def __getitem__(self, idx):


      return {'user': torch.tensor(, dtype=torch.long),
              'movie':  torch.tensor(, dtype=torch.long),
              'rating':  torch.tensor(, dtype=torch.float)}

  def __len__(self):
    return len()


# Create your DataLoaders

Now as in the labs, we want to fetch lot of data at the same time to send it to the model. We use DataLoaders. Define Dataloaders for each of your DataSets

# Create a Model

Create a model that inherits from nn.Module. A skeleton is given. If you don't have any ideas, try to reimplement NCF model.

Documenation available on : https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
class RecModel(nn.Module):


    def __init__(self,..):


    def forward(self, ..):

# Train

Form your training Loop and train your model. Don't forget your optimizer, batch size , number of epochs...

Documentation with example on : https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

# Test

Test your model on the test set. 
Do we need a metric ? If yes use the RSME as a metric